In [1]:
import pandas as pd
import json

In [5]:
def flatten_json(json_data, prefix=''):
    items = {}
    for k, v in json_data.items():
        new_key = f"{prefix}.{k}" if prefix else k
        if isinstance(v, dict):
            items.update(flatten_json(v, new_key))
        else:
            items[new_key] = v
    return items



In [1]:
from descarga_datos import *

In [4]:
token = get_token()

In [6]:
tasks = get_group_tasks(token, group_id="6730de789c42b399653929b6")
len(tasks)

128

In [11]:
tasks = [task.get("properties") for task in tasks if task.get("properties")]

In [12]:
task = tasks[0]
task

{'datosSotano.inundado': False,
 'datosSotano.deformacion.danos': False,
 'datosSotano.deformacion.actuacion': False,
 'datosSotano.fisuras.danos': False,
 'datosSotano.fisuras.actuacion': False,
 'datosSotano.desprendimientos.danos': False,
 'datosSotano.desprendimientos.actuacion': False,
 'datosSotano.humedadAmbiente.danos': False,
 'datosSotano.instalacionesSaneamientos.danos': False,
 'datosSotano.instalacionesAbastecimientos.danos': False,
 'datosSotano.instalacionesElectricidad.danos': False,
 'datosSotano.instalacionesGas.danos': False,
 'datosPlantaBaja.deformacion.danos': True,
 'datosPlantaBaja.deformacion.actuacion': False,
 'datosPlantaBaja.deformacion.zona': 'Elemento horizontal',
 'datosPlantaBaja.deformacion.comentarios': 'Hundimiento puntual pavimento planta baja',
 'datosPlantaBaja.fisuras.danos': False,
 'datosPlantaBaja.fisuras.actuacion': False,
 'datosPlantaBaja.desprendimientos.danos': False,
 'datosPlantaBaja.desprendimientos.actuacion': False,
 'datosPlantaBaja

In [13]:
completed_tasks = filter_tasks(tasks, status="FINISHED")
len(completed_tasks)

85

In [6]:
import json
import pandas as pd

In [4]:
groups = get_groups(token)

In [3]:
token = get_token()

In [ ]:
from datetime import datetime

all_data = []
# Obtener la fecha y hora actual en formato yyyy-mm-dd_HH-MM-SS
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename_csv = f"{current_datetime}.csv"
filename_json = f"{current_datetime}.json"
for group in groups:
    group_name = group['name']
    tasks = get_group_tasks(token, group['_id'])
    completed_tasks = filter_tasks(tasks, status='FINISHED')
    
    for completed_task in completed_tasks:
        # Extrae los campos adicionales directamente del task
        task_data = {
            'nombre_grupo': group_name,
            '_id': completed_task.get('_id'),
            'groupId': completed_task.get('groupId'),
            'missionId': completed_task.get('missionId'),
            'userId': completed_task.get('userId'),
            'layerId': completed_task.get('layerId'),
            'featureId': completed_task.get('featureId'),
            'status': completed_task.get('status'),
            'result': completed_task.get('result'),
            'statusChangedAt': completed_task.get('statusChangedAt'),
            'resultChangedAt': completed_task.get('resultChangedAt'),
            'formChangedAt': completed_task.get('formChangedAt'),
            'createdAt': completed_task.get('createdAt')
        }
        
        # Obtener y decodificar el JSON de `formData`
        stringJson = completed_task.get('formData')
        if not stringJson:
            continue
        json_data = json.loads(stringJson)
        
        # Aplanar los datos de `formData`
        flattened_formData = flatten_json(json_data)
        
        # Extraer la información de `feature` y `properties`
        feature = completed_task.get('feature', {})
        
        # Aplanar las propiedades de `properties` dentro de `feature`
        properties = feature.get('properties', {})
        flattened_properties = flatten_json(properties)
        
        # Combinar toda la información en un solo diccionario
        combined_data = {**task_data, **flattened_formData, **flattened_properties}
        all_data.append(combined_data)


df = pd.DataFrame(all_data)
df.to_csv(filename_csv, index=False)

In [22]:
df.head()

,nombre_grupo,datosSotano.inundado,datosSotano.deformacion.danos,datosSotano.deformacion.actuacion,datosSotano.fisuras.danos,datosSotano.fisuras.actuacion,datosSotano.desprendimientos.danos,datosSotano.desprendimientos.actuacion,datosSotano.humedadAmbiente.danos,datosSotano.instalacionesSaneamientos.danos,...,tecnicos.fechaInspeccion,tecnicos.nColegiadoTecnico1,tecnicos.tecnico1,tecnicos.nColegiadoTecnico2,tecnicos.tecnico2,datosResponsableZona.nombre,datosResponsableZona.telefono,datosPlantaBaja.espaciosNoOperativos.comentarios,datosSotano.deformacion.comentarios,datosPlantaBaja.accesibilidad.zona
0,MAPEO ALDAIA,False,False,False,False,False,False,False,False,False,...,2024-11-11T10:54:18.397Z,03506,Juan R. Navarro Martínez,03735,Fernando Olba Rallo,NaN,NaN,NaN,NaN,NaN
1,MAPEO CATARROJA,False,False,False,False,False,False,False,False,False,...,NaN,03793,Gonzalo Vicente Almazan,8110,Guillermo Navarro Mengual,Francisco J López Zarco,606997787,NaN,NaN,NaN
2,MAPEO CATARROJA,False,False,False,False,False,False,False,False,False,...,NaN,03793,Gonzalo Vicente Almazan,8110,Guillermo Navarro Mengual,Francisco J López Zarco,606997787,Vivienda planta baja,NaN,NaN
3,MAPEO CATARROJA,False,False,False,False,False,False,False,False,False,...,NaN,03793,Gonzalo Vicente Almazan,8110,Guillermo Navarro Mengual,Francisco J López Zarco,606997787,NaN,NaN,NaN
4,MAPEO CATARROJA,False,False,False,False,False,False,False,False,False,...,2024-11-11T10:20:10.572Z,03793,Gonzalo Vicente Almazan,8110,Guillermo Navarro Mengual,Francisco J López Zarco,606997787,NaN,,Foso
